# Weekly Meeting 12

Keypoints of the previous meeting:

* Going back to analyze the errors of the generative model.


* To sample candidates with the generative system + discriminative reranking [Collins, 2005]


* To use/extend the task for summarization.


* To check the robustness for larger rankings.

## Going back and analyze

I have written code for "easy" visualize and compare the results of the generative and discriminative systems:

http://localhost:8888/notebooks/GoBackAnalysis.ipynb

The behavior of the generative and the discriminative systems is very similar. In many cases, the generative system ranks high some coherent distractors (examples: 5, 63, 28, 13, 15, 1, 2, 7, 9, 11, ...). But others are quite intriguing (32, 34, 36, ...) and I haven't found any systematic error.

## Discriminative reranking

is the goal to generate and rerank candidates for unseen entity tuples? ("in the wild, the oracle selection of candidates is not possible").

If so, I'm not sure about how to evaluate if it is better to rerank the candidates or not, but visually it seems quite reasonable: http://localhost:8888/notebooks/GoBackAnalysis.ipynb (10, 9).)


**Simulating "the wild"**: use the gold aggregations of the test set only to evaluate, not for generating the list of candidates.

<img src="Wild-Evaluation.png">

\* The alignment seems quite easier than in the previous experiments where we checked for alignments between the ancestors and gold aggregations:

See example 10 and http://localhost:8888/notebooks/OverlappingGeneratorCandidates-Gold.ipynb (Levenshtein at word-level for the alignments). 
<br>**I computed the 24 more likely generations. The 62% of the gold aggregations appear exactly in the generations (0 edit distance). Also, with a maximum of 1 edit distance, the coverage increases to 87.27%. In this case, some errors appear (especially harmful on candidates with only 1 token) but these errors can approximate the gold aggregations in some cases: 'connecticut democratic politicians': 'connecticut politicians'. With a maximum edit distance of 1, only for the aggregations of length >1, the coverage is 79%**



| System | Coverage (0 dist) | Coverage (<=1 dist) | Coverage (0 dist or (<=1 dist for >1 token)) |
| --- | --- | --- | --- |
| Generative-1 | 60.00% | 86.40% | 77.74% |
| Generative-2 | 62.16% | 87.27% | 79.13% |


Results of the wild evaluation (OUTDATES: there was a bug in the alignments, See WeeklyMeeting14):


| Ranking System | Alignment | MAP | R@10 | MRR |
| --- | --- | --- | --- | --- |
| Generative-2 | 0 dist | 59.78 | 88.32 | 67.43 |
| Generative-2 | <=1 dist | 58.71 | 84.44 | 68.82 |
| Generative-2 | <=1 dist for >1 token | 59.19 | 85.37 | 69.00 |
| Discriminative | 0 dist | 54.02 | 86.03 | 60.52 |
| Discriminative | <=1 dist | 52.40 | 81.10 | 61.57 |
| Discriminative | <=1 dist for >1 token | 51.23 | 81.73 | 59.64 |

## Summarization

https://catalog.ldc.upenn.edu/docs/LDC2008T19/new_york_times_annotated_corpus.pdf

Following the Jackie's idea from the previous meeting: to build a corpus for summarization by asking for sentence completion given the aggregation of the entities, I've thought about two different summarization tasks:

* **Semantic context summarization**: summarization of spans (contexts) that mention entities together.

> **Context**: <span style="color:purple">Street Violence by Paris Youths Intrudes Again into French Politics:</span><span style="color:blue"> The Socialist candidate, <span style="color:red">Segolene Royal</span>, who is running second in the opinion polls, said the incident showed that <span style="color:red">Mr. Sarkozy</span> had failed as interior minister. '' In five years with a right-wing government that has made crime its main campaign issue, you can see that it is a failure all the way,'' she said on Canal+ television. <span style="color:red">Francois Bayrou</span>, a centrist presidential candidate, also took aim at <span style="color:red">Mr. Sarkozy</span>, saying,'' It is very important to end this climate of perpetual confrontation between police and some citizens.''</span>
<br><br>**Summary**: The <span style="color:orange">[french politicians | politicians | french presidential candidates]</span> <span style="color:green">have been involved in a dialectical confrontation.</span>

> **Context**: <span style="color:purple">Battleground For Consoles Moves Online:</span><span style="color:blue"> Over all, though, it is <span style="color:red">Microsoft</span> that has had the steeper mountain to climb. In the last generation of video game consoles, <span style="color:red">Sony</span> had a roughly 60 percent market share, compared to 20 percent for each Microsoft and Nintendo.</span>
  <br><br>**Summary**: The <span style="color:orange">[technology companies | multinational corporations]</span><span style="color:green"> compete for the market share of the new generation of consoles.</span> 
  
* **Semantic document summarization**: summarization of full articles. As some of the summaries of the NYT corpus mention entities together, they can be rewritten in terms of the aggregations.

> **Context**: <span style="color:purple">Street Violence by Paris Youths Intrudes Again into French Politics:</span><span style="color:blue"> [...] The Socialist candidate, <span style="color:red">Segolene Royal</span>, who is running second in the opinion polls, said the incident showed that <span style="color:red">Mr. Sarkozy</span> had failed as interior minister. '' In five years with a right-wing government that has made crime its main campaign issue, you can see that it is a failure all the way,'' she said on Canal+ television. <span style="color:red">Francois Bayrou</span>, a centrist presidential candidate, also took aim at <span style="color:red">Mr. Sarkozy</span>, saying,'' It is very important to end this climate of perpetual confrontation between police and some citizens.'' [...]</span>
<br><br>**Original summary**: <span style="color:red">Francois Bayrou, Segolene Royal and Nicolas Sarkozy</span><span style="color:green"> seized on a new campaign issue on Wednesday after the police fought gangs of youths for seven hours at a railroad station here. Stores were destroyed, the pall of tear gas filled the station, Gare du Nord, and 13 people were arrested.</span>
<br><br>**Semantic summary (1)**: The <span style="color:orange">[french presidential candidates | politicians | french politicians]</span> <span style="color:green">seized on a new campaign issue on Wednesday after the police fought gangs of youths for seven hours at a railroad station here. Stores were destroyed, the pall of tear gas filled the station, Gare du Nord, and 13 people were arrested.</span>
<br><br>**Semantic summary (2)**: The <span style="color:orange">[french presidential candidates | politicians | french politicians]</span> <span style="color:green">seized on a new campaign issue on Wednesday after incidents caused by gangs of youths in Gare du Nord.</span>


Both of them can be annotated in the same way, asking for summarizing short spans of sentences that contain the entities. The only difference is that for "Semantic context summarization" all the spans that mention entities together can be used (both from the articles and summaries) but for "Semantic document summarization" only the summaries that mention entities can be used (we can only be sure that a semantic aggregation of entities is relevant for a summary of the full article if these entities appear in the summary).

A sorted list of possible aggregations, extracted with the generative system + discriminative reranking, can be given to the annotators (if none of these aggregation is well suitted for the entities, discard or ask the user to provide one). They are asked for summarize the short span of sentences that mention the entities in terms of the aggregation they selected.

<img src="form.jpg">

## Checking the robustness for larger rankings

**Experiment 1**: Build rankings of the maximum possible size with the same type aggregations of the corpus. The ranking size must be the same size for all tuples, but the size of the aggregation set of each entity type differs (951 Person, 505 Location, 239 Org), so the ranking size is the minimum of this: 239.

**Experiment 2**: Increase the ranking size by extracting negative candidates from the Wikipedia category graph. Each ranking of size R is composed by:

1) All the aggregations of the same type. 

2) [If the ranking has not been filled with R candidates] Common ancestors (typically thousands) sorted by sum of distances.

3) [If the ranking has not been filled with R candidates] Union of direct neighbors of each entity in the tuple.

4) [If the ranking has not been filled with R candidates] Random nodes from the graph.

If some entity of a tuple does not appear in the graph (~20% tuples), random nodes are used. Visually, lots of candidates extracted from the graph aren't of the same type than the entity tuple ['Bernard B. Kerik', 'Rudolph W. Giuliani'] $\rightarrow$ ['populated places in belgium'] (possibly due to I picked all the lowest common ancestors until filling R candidates).

<br>

| Training RS | Test RS | Candidates source | MAP | R@10 | MRR |
| --- | --- | --- | --- | --- | --- |
| 24 | 24 | Tesa | **89.80** | **99.28** | **95.69** |
| 24 | 239 | Tesa | 68.75 | 88.52 | 81.84 |
| 24 | 2000 | Tesa + Category graph | 47.29 | 70.33 | 63.60 |
| 239 | 239 | Tesa | 58.80 | 77.85 | 76.60 |
| 2000 | 2000 | Tesa + Category graph | - | - | - |